# SSFP Two-Pool Steady State (Updated $R_x$, $R_z$)

This notebook derives the steady-state readout for a two-pool (exchange) SSFP model using **updated rotation conventions**.

**Outline**
1. State the model and updated rotations.
2. Build $\mathbf{P} = \mathbf{A}\mathbf{E} R_z R_x$ and $\mathbf{Y} = \mathbf{A}\mathbf{B}$ using sparsity.
3. Solve $(\mathbf{I}-\mathbf{P})\,\mathbf{m}^- = \mathbf{Y}$ via two $2\times2$ solves.
4. Form the complex readout $m^+$ with the updated $R_x$.
5. Present the target SSFP form and $\beta$-factorization; (optional) numeric sanity checks.


## Updated rotations

We use the following updated rotation matrices:

$$R_x(\alpha) = \begin{bmatrix}
1 & 0 & 0 & 0\\
0 & \cos\alpha & \sin\alpha & 0\\
0 & -\sin\alpha & \cos\alpha & 0\\
0 & 0 & 0 & f_w
\end{bmatrix}$$

$$R_z(\varphi) = \begin{bmatrix}
\cos\varphi & \sin\varphi & 0 & 0 \\
-\sin\varphi & \cos\varphi & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 
\end{bmatrix}$$


## $\beta$-parameters and target form

We factor the final expressions using:

$$\beta_1 = 1 + F - f_w E_{1,r} (F + f_k)$$
$$\beta_2 = 1 + f_k \left( F - f_w E_{1,r}(F+1)\right)$$
$$\beta_3 = E_{2}\left(\beta_2 E_{1,f} -  \beta_1 \right) \left( 1 + \cos \alpha \right)$$
$$\beta_4 =  \beta_1  - \beta_2 E_{1,f} \cos \alpha - \left(\beta_2 E_{1,f} -  \beta_1  \cos \alpha\right)E_{2}^2$$
$$\beta_5 = M_{0,f} \beta_2 \left(1 - E_{1,f}\right) + M_{0,r}\left(1 - E_{1,r}\right)\left(1 - f_k\right)$$

The target SSFP form is

$$m^+ = A\,\frac{1 - E_2 e^{i\varphi}}{B\cos\varphi + C}, \quad \text{with } A\propto\beta_5,\; B\propto\beta_3,\; C\propto\beta_4.$$


In [ ]:
# Imports and symbols
import sympy as sp
E2, E1f, E1r, fw = sp.symbols('E2 E1f E1r fw', real=True)
F, fk            = sp.symbols('F fk', real=True)
ca, sa           = sp.symbols('ca sa', real=True)      # cos(alpha), sin(alpha)
cphi, sphi       = sp.symbols('cphi sphi', real=True)  # cos(phi), sin(phi)
M0f, M0r         = sp.symbols('M0f M0r', real=True)

# a,b,c,d in terms of F,fk (kept symbolic for structure)
a = (1 + F*fk)/(F + 1)
b = (1 - fk)/(F + 1)
c = (F*(1 - fk))/(F + 1)
d = (F + fk)/(F + 1)

a, b, c, d

## Build $\mathbf{P} = \mathbf{A}\mathbf{E} R_z R_x$ and $\mathbf{Y}=\mathbf{A}\mathbf{B}$

<!-- \(\require{color}\) -->
Let $c_\alpha=\cos\alpha$, $s_\alpha=\sin\alpha$, $c_\varphi=\cos\varphi$, $s_\varphi=\sin\varphi$. With the substitutions $a$, $b$, $c$, and $d$ above, the matrix product $\mathbf{P} = \mathbf{A}\mathbf{E} R_z R_x$ is given by 
$$\mathbf{P} = \mathbf{A}\mathbf{E}R_z R_x =
\begin{bmatrix}
\color{#1f77b4}{E_2 c_\varphi} & \color{#1f77b4}{E_2 c_\alpha s_\varphi} & \color{#9467bd}{E_2 s_\alpha s_\varphi} & \color{#888888}{0} \\
\color{#1f77b4}{-E_2 s_\varphi} & \color{#1f77b4}{E_2 c_\alpha c_\varphi} & \color{#9467bd}{E_2 s_\alpha c_\varphi} & \color{#888888}{0} \\
\color{#888888}{0} & \color{#ff7f0e}{-a E_{1,f} s_\alpha} & \color{#2ca02c}{a E_{1,f} c_\alpha} & \color{#2ca02c}{b E_{1,r} f_w} \\
\color{#888888}{0} & \color{#ff7f0e}{-c E_{1,f} s_\alpha} & \color{#2ca02c}{c E_{1,f} c_\alpha} & \color{#2ca02c}{d E_{1,r} f_w}
\end{bmatrix}. $$



Define the transverse $2\times2$ block and the $m_{z,f}\to (m_x,m_y)$ drive:

$$\mathbf{M}_x = \begin{bmatrix} E_2 c_\varphi & E_2 c_\alpha s_\varphi \\ -E_2 s_\varphi & E_2 c_\alpha c_\varphi\end{bmatrix},\quad
\mathbf{A}_{13} = \begin{bmatrix} E_2 s_\alpha s_\varphi \\ E_2 s_\alpha c_\varphi \end{bmatrix}.$$

The longitudinal block (after including $\mathbf{A}$ and $\mathbf{E}$) is parameter-only:

$$\begin{aligned}
(\mathbf{I}-\mathbf{P})_{\text{long}} &= \begin{bmatrix}
1 - a E_{1,f} c_\alpha & - b E_{1,r} f_w \\
- c E_{1,f} c_\alpha & 1 - d E_{1,r} f_w
\end{bmatrix},\\[4pt]
\mathbf{Y} &= \begin{bmatrix} 0 \\ 0 \\ a M_{0,f}(1-E_{1,f}) + b M_{0,r}(1-E_{1,r}) \\ c M_{0,f}(1-E_{1,f}) + d M_{0,r}(1-E_{1,r})\end{bmatrix}.
\end{aligned}$$

**Full matrix $\mathbf{P}$ (color-coded):**

- \(\color{#1f77b4}{\text{blue}}\): transverse block $\mathbf{M}_x$ (top-left $2\times2$)
- \(\color{#9467bd}{\text{purple}}\): coupling column $\mathbf{A}_{13}$ into $(m_x,m_y)$ from $m_{z,f}$
- \(\color{#ff7f0e}{\text{orange}}\): longitudinal coupling from $m_y$ into $m_{z,\cdot}$
- \(\color{#2ca02c}{\text{green}}\): longitudinal $(z_f,z_r)$ propagation (exchange/relaxation)
- \(\color{#888888}{\text{gray}}\): structural zeros




In [ ]:
# Transverse 2x2 block and the coupling column
Mx  = sp.Matrix([[E2*cphi,      E2*ca*sphi],
                 [-E2*sphi,     E2*ca*cphi]])
A13 = sp.Matrix([E2*sa*sphi,  E2*sa*cphi])

# Longitudinal (I - P)_long and Y = A B
A_low = sp.Matrix([[1 - a*E1f*ca,   -b*E1r*fw],
                   [-c*E1f*ca,      1 - d*E1r*fw]])
y3 = a*M0f*(1 - E1f) + b*M0r*(1 - E1r)
y4 = c*M0f*(1 - E1f) + d*M0r*(1 - E1r)
A_low, y3, y4

## Blockwise solution

\(\require{color}\)
We solve the transverse subsystem first, then the longitudinal one, and finally back-substitute:

- Transverse (blue/purple):

$$(\mathbf{I}_2 - \color{#1f77b4}{\mathbf{M}_x})\begin{bmatrix}m_x\\ m_y\end{bmatrix}
 = \color{#9467bd}{\mathbf{A}_{13}}\, m_{z,f}.$$

- Longitudinal (orange/green): move the $m_y$ coupling to the left so the right-hand side is just $\mathbf{Y}$.


In [ ]:
# Transverse solve gives [mx; my] = U * mzf
I2 = sp.eye(2)
mzf_sym = sp.Symbol('mzf')
u_sol = (I2 - Mx).LUsolve(A13 * mzf_sym)
mx_lin, my_lin = u_sol[0], u_sol[1]

# Extract linear coefficients U (to avoid carrying a temporary symbol)
mx_coef = sp.together(mx_lin / mzf_sym)
my_coef = sp.together(my_lin / mzf_sym)
mx_coef, my_coef

### Longitudinal system with $m_y$ moved left

\(\require{color}\)
Using $m_y = \text{my\_coef}\,m_{z,f}$, the longitudinal system becomes

$$\underbrace{\begin{bmatrix}
\color{#2ca02c}{1 - aE_{1,f}c_\alpha} \; + \; \color{#ff7f0e}{aE_{1,f}s_\alpha\,\text{my\_coef}} & \color{#2ca02c}{- bE_{1,r} f_w}\\
\color{#2ca02c}{-cE_{1,f}c_\alpha} \; + \; \color{#ff7f0e}{cE_{1,f}s_\alpha\,\text{my\_coef}} & \color{#2ca02c}{1 - dE_{1,r} f_w}
\end{bmatrix}}_{\mathbf{A}_{\text{long}}}
\begin{bmatrix} m_{z,f}\\ m_{z,r}\end{bmatrix} 
= \begin{bmatrix} y_3\\ y_4\end{bmatrix}.$$

Here, **orange** terms are the longitudinal coupling induced by $m_y$ (via $s_\alpha$), and **green** terms are the exchange/relaxation propagation.


In [ ]:
# Build and solve the corrected longitudinal system
A_long = sp.Matrix([[1 - a*E1f*ca + a*E1f*sa*my_coef,   -b*E1r*fw],
                    [-c*E1f*ca + c*E1f*sa*my_coef,       1 - d*E1r*fw]])
rhs_long = sp.Matrix([y3, y4])
mzf_expr, mzr_expr = A_long.LUsolve(rhs_long)

# Back-substitute for mx,my
mx_ss = sp.together(mx_coef * mzf_expr)
my_ss = sp.together(my_coef * mzf_expr)
mzf_expr, mx_ss, my_ss

## Readout $m^+$ (updated $R_x$)

With $x^+=m_x$, $y^+=\cos\alpha\,m_y + \sin\alpha\,m_{z,f}$, we form

$$m^+ = m_x + i\,(\cos\alpha\,m_y + \sin\alpha\,m_{z,f}).$$


In [ ]:
# Complex readout
mplus = sp.together(mx_ss + sp.I*(ca*my_ss + sa*mzf_expr))
mplus

## $\beta$-parameters and target SSFP form

We define the $\beta$-parameters and target the form

$$m^+ = A\,\frac{1 - E_2 e^{i\varphi}}{B\cos\varphi + C},\quad \text{with } A\propto\beta_5,\; B\propto\beta_3,\; C\propto\beta_4.$$


In [ ]:
# β-blocks in (F,fk) form
beta1 = 1 + F - fw*E1r*(F + fk)
beta2 = 1 + fk*(F - fw*E1r*(F + 1))
beta3 = E2*((beta2*E1f - beta1))*(1 + ca)
beta4 = beta1 - beta2*E1f*ca - (beta2*E1f - beta1*ca)*E2**2
beta5 = M0f*beta2*(1 - E1f) + M0r*(1 - E1r)*(1 - fk)
beta1, beta2, beta3, beta4, beta5

## Optional: numeric sanity checks

We can test the identity (up to a global readout phase)

$$m^+ \stackrel{?}{=} (\beta_5\,\sin\alpha)\,\frac{1 - E_2 e^{i\varphi}}{\beta_3\cos\varphi + \beta_4}$$

across random parameter draws.

In [ ]:
import mpmath as mp, random, math

vars_list = (E2,E1f,E1r,fw,F,fk,ca,sa,cphi,sphi,M0f,M0r)
mplus_fun = sp.lambdify(vars_list, mplus, 'mpmath')
beta3_fun = sp.lambdify((E2,E1f,E1r,fw,F,fk,ca), beta3, 'mpmath')
beta4_fun = sp.lambdify((E1r,fw,F,fk,E1f,E2,ca), beta4, 'mpmath')
beta5_fun = sp.lambdify((M0f,M0r,E1f,E1r,F,fk,fw), beta5, 'mpmath')

def rand_params():
    E2v  = random.uniform(0.3, 0.95)
    E1fv = random.uniform(0.7, 0.99)
    E1rv = random.uniform(0.7, 0.99)
    fwv  = random.uniform(0.8, 1.0)
    Fv   = random.uniform(0.2, 5.0)
    fkv  = random.uniform(0.0, 0.95)
    alpha = random.uniform(0.05, 1.45)
    phi   = random.uniform(-2.8, 2.8)
    ca_v, sa_v     = math.cos(alpha), math.sin(alpha)
    cphi_v, sphi_v = math.cos(phi), math.sin(phi)
    M0fv = random.uniform(0.8, 1.2)
    M0rv = random.uniform(0.8, 1.2)
    return dict(E2=E2v, E1f=E1fv, E1r=E1rv, fw=fwv, F=Fv, fk=fkv,
                ca=ca_v, sa=sa_v, cphi=cphi_v, sphi=sphi_v, M0f=M0fv, M0r=M0rv)

def trial():
    s = rand_params()
    mval = mplus_fun(s['E2'],s['E1f'],s['E1r'],s['fw'],s['F'],s['fk'],
                     s['ca'],s['sa'],s['cphi'],s['sphi'],s['M0f'],s['M0r'])
    B = beta3_fun(s['E2'],s['E1f'],s['E1r'],s['fw'],s['F'],s['fk'],s['ca'])
    C = beta4_fun(s['E1r'],s['fw'],s['F'],s['fk'],s['E1f'],s['E2'],s['ca'])
    A_sin = beta5_fun(s['M0f'],s['M0r'],s['E1f'],s['E1r'],s['F'],s['fk'],s['fw']) * s['sa']
    num = 1 - s['E2']*(s['cphi'] + 1j*s['sphi'])
    den = B*s['cphi'] + C
    if abs(num) < 1e-9 or abs(den) < 1e-9 or abs(A_sin) < 1e-12:
        return None
    A_est = mval * den / num
    ratio  = A_est / A_sin
    ratioi = A_est / (1j * A_sin)
    return (float(abs(abs(ratio)-1)), float(mp.arg(ratio)),
            float(abs(abs(ratioi)-1)), float(mp.arg(ratioi)))

trials = [trial() for _ in range(5)]
trials